In [383]:
import uproot
import uproot_methods.classes.TH2
import numpy as np
import pandas as pd
import math

In [3]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
from scipy.signal import savgol_filter

In [4]:
def integral(array,integration_interval,x_domain,nbin):
    s=0
    Δx=x_domain[1]-x_domain[0]
    #ϵ=abs(Δx)/nbin
    i_min=int(round(nbin*(integration_interval[0]-x_domain[0])/Δx))
    i_max=int(round(nbin*(integration_interval[1]-x_domain[0])/Δx))
    
    for i in range(i_min,i_max):
        s=array[i]+s
    return s

In [5]:
#### WORKS ONLY FOR MONOTONIC FUNCTIONS (INVERTIBLE FUNCTIONS) ####
#### FOR INCREASING F's, SET sgn=+1 ####
#### FOR DECREASING F's, SET sgn=-1 ####
def inverse(y,f,interval,sgn):
    if y<0.0 or y>1.0:
        return 'nan';
    ϵ=1e-7
    ϵ2=1e-7
    x_min=interval[0]
    x_max=interval[1]
    Δx=x_max-x_min
    q=1;n=0;
#    print(q, x_min, x_max)
    while abs(q)>ϵ and abs(Δx)>ϵ2:
#        x0=unif(x_min,x_max)    
        x0=0.5*(x_max+x_min)
            
        t=f(x0)
        q=t-y
        if sgn*q<=0:
            x_min=x0
        if sgn*q>0:
            x_max=x0
            
        Δx=x_max-x_min
        n=n+1
#        print(x0, t, sgn*q, [x_min, x_max])
    return x0,q,n

In [6]:
class algs:
    def __init__(self,path,file_name,algos):#algos=[ip2d,ip3d,rnnip,sv1,jf,DL1]

        file_empf201903=uproot.open(path+file_name)
        n_B=np.array(file_empf201903["nBjets"]).sum()
        n_C=np.array(file_empf201903["nCjets"]).sum()
        n_l=np.array(file_empf201903["nljets"]).sum() 
        
        empf201903_ip2d_exB=np.array(file_empf201903["ip2d_llr_B"])
        empf201903_ip2d_exC=np.array(file_empf201903["ip2d_llr_C"])
        empf201903_ip2d_l=np.array(file_empf201903["ip2d_llr_l"])
        #print(len(empf201903_ip2d_exB)==len(empf201903_ip2d_l),len(empf201903_ip2d_l))
        empf201903_ip2d_exB=np.delete(empf201903_ip2d_exB, [0,len(empf201903_ip2d_exB)-1])
        empf201903_ip2d_exC=np.delete(empf201903_ip2d_exC, [0,len(empf201903_ip2d_exC)-1])
        empf201903_ip2d_l=np.delete(empf201903_ip2d_l, [0,len(empf201903_ip2d_l)-1])

        empf201903_ip3d_exB=np.array(file_empf201903["ip3d_llr_B"])
        empf201903_ip3d_exC=np.array(file_empf201903["ip3d_llr_C"])
        empf201903_ip3d_l=np.array(file_empf201903["ip3d_llr_l"])
        #print(len(empf201903_ip3d_exB)==len(empf201903_ip3d_l),len(empf201903_ip3d_l))
        empf201903_ip3d_exB=np.delete(empf201903_ip3d_exB, [0,len(empf201903_ip3d_exB)-1])
        empf201903_ip3d_exC=np.delete(empf201903_ip3d_exC, [0,len(empf201903_ip3d_exC)-1])
        empf201903_ip3d_l=np.delete(empf201903_ip3d_l, [0,len(empf201903_ip3d_l)-1])

        empf201903_rnnip_exB=np.array(file_empf201903["rnnip_llr_B"])
        empf201903_rnnip_exC=np.array(file_empf201903["rnnip_llr_C"])
        empf201903_rnnip_l=np.array(file_empf201903["rnnip_llr_l"])
        #print(len(empf201903_rnnip_exB)==len(empf201903_rnnip_l),len(empf201903_rnnip_l))
        empf201903_rnnip_exB=np.delete(empf201903_rnnip_exB, [0,len(empf201903_rnnip_exB)-1])
        empf201903_rnnip_exC=np.delete(empf201903_rnnip_exC, [0,len(empf201903_rnnip_exC)-1])
        empf201903_rnnip_l=np.delete(empf201903_rnnip_l, [0,len(empf201903_rnnip_l)-1])

        empf201903_sv1_exB=np.array(file_empf201903["sv1_llr_B"])
        empf201903_sv1_exC=np.array(file_empf201903["sv1_llr_C"])
        empf201903_sv1_l=np.array(file_empf201903["sv1_llr_l"])
        #print(len(empf201903_sv1_exB)==len(empf201903_sv1_l),len(empf201903_sv1_exB))
        empf201903_sv1_exB=np.delete(empf201903_sv1_exB, [0,len(empf201903_sv1_exB)-1])
        empf201903_sv1_exC=np.delete(empf201903_sv1_exC, [0,len(empf201903_sv1_exC)-1])
        empf201903_sv1_l=np.delete(empf201903_sv1_l, [0,len(empf201903_sv1_l)-1])            

        empf201903_jf_exB=np.array(file_empf201903["jf_llr_B"])
        empf201903_jf_exC=np.array(file_empf201903["jf_llr_C"])
        empf201903_jf_l=np.array(file_empf201903["jf_llr_l"])
        #print(len(empf201903_jf_exB)==len(empf201903_jf_l),len(empf201903_jf_exB))
        empf201903_jf_exB=np.delete(empf201903_jf_exB, [0,len(empf201903_jf_exB)-1])
        empf201903_jf_exC=np.delete(empf201903_jf_exC, [0,len(empf201903_jf_exC)-1])
        empf201903_jf_l=np.delete(empf201903_jf_l, [0,len(empf201903_jf_l)-1])            

        empf201903_dl1_exB=np.array(file_empf201903["dl1_llr_B"])
        empf201903_dl1_exC=np.array(file_empf201903["dl1_llr_C"])
        empf201903_dl1_l=np.array(file_empf201903["dl1_llr_l"])
        #print(len(empf201903_dl1_exB)==len(empf201903_dl1_l),len(empf201903_dl1_exB))
        empf201903_dl1_exB=np.delete(empf201903_dl1_exB, [0,len(empf201903_dl1_exB)-1])
        empf201903_dl1_exC=np.delete(empf201903_dl1_exC, [0,len(empf201903_dl1_exC)-1])
        empf201903_dl1_l=np.delete(empf201903_dl1_l, [0,len(empf201903_dl1_l)-1])

        nbin=len(empf201903_ip2d_exB)
        x_min=-20
        x_max=50
        xaxis=[x_min,x_max]
        Δx=x_max-x_min
        interval=Δx*(np.arange(0,nbin)/nbin)+x_min
        #print(len(interval)==nbin,nbin)

        nbin_dl1=len(empf201903_dl1_exB)
        xdl1_min=-20
        xdl1_max=50
        xdl1axis=[xdl1_min,xdl1_max]
        Δx_dl1=xdl1_max-xdl1_min
        interval_dl1=Δx_dl1*(np.arange(0,nbin_dl1)/nbin_dl1)+xdl1_min
        #print(len(interval_dl1)==nbin_dl1,nbin_dl1)

        empf201903_N_l=integral(empf201903_ip2d_l,xaxis,xaxis,nbin)
        empf201903_N_exB=integral(empf201903_ip2d_exB,xaxis,xaxis,nbin)
        empf201903_N_exC=integral(empf201903_ip2d_exC,xaxis,xaxis,nbin)

        empf201903_N_ip3dl=integral(empf201903_ip3d_l,xaxis,xaxis,nbin)
        empf201903_N_ip3dexB=integral(empf201903_ip3d_exB,xaxis,xaxis,nbin)
        empf201903_N_ip3dexC=integral(empf201903_ip3d_exC,xaxis,xaxis,nbin)

        empf201903_N_rnnipl=integral(empf201903_rnnip_l,xaxis,xaxis,nbin)
        empf201903_N_rnnipexB=integral(empf201903_rnnip_exB,xaxis,xaxis,nbin)
        empf201903_N_rnnipexC=integral(empf201903_rnnip_exC,xaxis,xaxis,nbin)

        empf201903_N_sv1l=integral(empf201903_sv1_l,xaxis,xaxis,nbin)
        empf201903_N_sv1exB=integral(empf201903_sv1_exB,xaxis,xaxis,nbin)
        empf201903_N_sv1exC=integral(empf201903_sv1_exC,xaxis,xaxis,nbin)

        empf201903_N_jfl=integral(empf201903_jf_l,xaxis,xaxis,nbin)
        empf201903_N_jfexB=integral(empf201903_jf_exB,xaxis,xaxis,nbin)
        empf201903_N_jfexC=integral(empf201903_jf_exC,xaxis,xaxis,nbin)

        empf201903_N_dl1l=integral(empf201903_dl1_l,xdl1axis,xdl1axis,nbin_dl1)
        empf201903_N_dl1exB=integral(empf201903_dl1_exB,xdl1axis,xdl1axis,nbin_dl1)
        empf201903_N_dl1exC=integral(empf201903_dl1_exC,xdl1axis,xdl1axis,nbin_dl1)

        xM_ip2d_B=empf201903_N_exB/n_B
        xM_ip3d_B=empf201903_N_ip3dexB/n_B
        xM_rnnip_B=empf201903_N_rnnipexB/n_B
        xM_sv1_B=empf201903_N_sv1exB/n_B
        xM_jf_B=empf201903_N_jfexB/n_B
        xM_dl1_B=empf201903_N_dl1exB/n_B
        xM_ip2d_C=empf201903_N_exC/n_C
        xM_ip3d_C=empf201903_N_ip3dexC/n_C
        xM_rnnip_C=empf201903_N_rnnipexC/n_C
        xM_sv1_C=empf201903_N_sv1exC/n_C
        xM_jf_C=empf201903_N_jfexC/n_C
        xM_dl1_C=empf201903_N_dl1exC/n_C
        xM_ip2d_l=empf201903_N_l/n_l
        xM_ip3d_l=empf201903_N_ip3dl/n_l
        xM_rnnip_l=empf201903_N_rnnipl/n_l
        xM_sv1_l=empf201903_N_sv1l/n_l
        xM_jf_l=empf201903_N_jfl/n_l
        xM_dl1_l=empf201903_N_dl1l/n_l
        print('Maximum efficiency \epsilon: ip2d, ip3d, rnnip, sv1, jf, dl1')
        print(xM_ip2d_B,xM_ip3d_B,xM_rnnip_B,xM_sv1_B,xM_jf_B,xM_dl1_B)
        print(xM_ip2d_C,xM_ip3d_C,xM_rnnip_C,xM_sv1_C,xM_jf_C,xM_dl1_C)        
        print(xM_ip2d_l,xM_ip3d_l,xM_rnnip_l,xM_sv1_l,xM_jf_l,xM_dl1_l)

        x_ax=np.array([])

        for i in range(0,nbin):

            x=i/nbin
            x_ax=np.append(x_ax,x)


        cut_v=np.array([])
        #cut_inverse=np.array([])

        empf201903_rocip2d_rej=np.array([])
        empf201903_rocip2d_rej_c=np.array([])

        empf201903_rocip3d_rej=np.array([])
        empf201903_rocip3d_rej_c=np.array([])

        empf201903_rocrnnip_rej=np.array([])
        empf201903_rocrnnip_rej_c=np.array([])

        empf201903_rocsv1_rej=np.array([])
        empf201903_rocsv1_rej_c=np.array([])

        empf201903_rocjf_rej=np.array([])
        empf201903_rocjf_rej_c=np.array([])

        empf201903_rocdl1_rej=np.array([])
        empf201903_rocdl1_rej_c=np.array([])

        for x in x_ax:
            if x<xM_ip2d_B:
                cut=inverse(x,lambda t: integral(empf201903_ip2d_exB/n_B,[t,x_max],xaxis,nbin),xaxis,-1)[0]
                cut_v=np.append(cut_v,cut)
                y=integral(empf201903_ip2d_l/n_l,[cut,x_max],xaxis,nbin)
                y_c=integral(empf201903_ip2d_exC/n_C,[cut,x_max],xaxis,nbin)
                empf201903_rocip2d_rej=np.append(empf201903_rocip2d_rej,1./y)
                empf201903_rocip2d_rej_c=np.append(empf201903_rocip2d_rej_c,1./y_c)
                
            if x<xM_ip3d_B:            
                cut=inverse(x,lambda t: integral(empf201903_ip3d_exB/n_B,[t,x_max],xaxis,nbin),xaxis,-1)[0]
                y=integral(empf201903_ip3d_l/n_l,[cut,x_max],xaxis,nbin)
                y_c=integral(empf201903_ip3d_exC/n_C,[cut,x_max],xaxis,nbin)
                empf201903_rocip3d_rej=np.append(empf201903_rocip3d_rej,1./y)
                empf201903_rocip3d_rej_c=np.append(empf201903_rocip3d_rej_c,1./y_c)

            if x<xM_rnnip_B:
                cut=inverse(x,lambda t: integral(empf201903_rnnip_exB/n_B,[t,x_max],xaxis,nbin),xaxis,-1)[0]
                y=integral(empf201903_rnnip_l/n_l,[cut,x_max],xaxis,nbin)
                y_c=integral(empf201903_rnnip_exC/n_C,[cut,x_max],xaxis,nbin)
                empf201903_rocrnnip_rej=np.append(empf201903_rocrnnip_rej,1./y)
                empf201903_rocrnnip_rej_c=np.append(empf201903_rocrnnip_rej_c,1./y_c)  

            if x<xM_sv1_B:
                cut=inverse(x,lambda t: integral(empf201903_sv1_exB/n_B,[t,x_max],xaxis,nbin),xaxis,-1)[0]
                y=integral(empf201903_sv1_l/n_l,[cut,x_max],xaxis,nbin)
                y_c=integral(empf201903_sv1_exC/n_C,[cut,x_max],xaxis,nbin)
                empf201903_rocsv1_rej=np.append(empf201903_rocsv1_rej,1./y)
                empf201903_rocsv1_rej_c=np.append(empf201903_rocsv1_rej_c,1./y_c)     

            if x<xM_jf_B:
                cut=inverse(x,lambda t: integral(empf201903_jf_exB/n_B,[t,x_max],xaxis,nbin),xaxis,-1)[0]
                y=integral(empf201903_jf_l/n_l,[cut,x_max],xaxis,nbin)
                y_c=integral(empf201903_jf_exC/n_C,[cut,x_max],xaxis,nbin)
                empf201903_rocjf_rej=np.append(empf201903_rocjf_rej,1./y)
                empf201903_rocjf_rej_c=np.append(empf201903_rocjf_rej_c,1./y_c)     

            if x<xM_dl1_B:
                cut=inverse(x,lambda t: integral(empf201903_dl1_exB/n_B,[t,xdl1_max],xdl1axis,nbin_dl1),xdl1axis,-1)[0]    
                y=integral(empf201903_dl1_l/n_l,[cut,xdl1_max],xdl1axis,nbin_dl1)
                y_c=integral(empf201903_dl1_exC/n_C,[cut,xdl1_max],xdl1axis,nbin_dl1)
                empf201903_rocdl1_rej=np.append(empf201903_rocdl1_rej,1./y)
                empf201903_rocdl1_rej_c=np.append(empf201903_rocdl1_rej_c,1./y_c)

            print(x, end='\r')

        pol_order=1
        den=20
        w_bin=int(len(empf201903_rocip2d_rej)/den)

        if(int(w_bin%2)==0):
            q=+1
        if(int(w_bin%2)!=0):
            q=-2

        windows_size=w_bin+q
        print(windows_size)

        if algos[0]==1:
            rej_ip2d = savgol_filter(empf201903_rocip2d_rej, windows_size, pol_order)
            rej_ip2d_c = savgol_filter(empf201903_rocip2d_rej_c, windows_size, pol_order)
        if algos[1]==1:
            rej_ip3d = savgol_filter(empf201903_rocip3d_rej, windows_size, pol_order)
            rej_ip3d_c = savgol_filter(empf201903_rocip3d_rej_c, windows_size, pol_order)
        if algos[2]==1:
            rej_rnnip = savgol_filter(empf201903_rocrnnip_rej, windows_size, pol_order)
            rej_rnnip_c = savgol_filter(empf201903_rocrnnip_rej_c, windows_size, pol_order)
        if algos[3]==1:
            rej_sv1 = savgol_filter(empf201903_rocsv1_rej, windows_size, pol_order)
            rej_sv1_c = savgol_filter(empf201903_rocsv1_rej_c, windows_size, pol_order)
        if algos[4]==1:
            rej_jf = savgol_filter(empf201903_rocjf_rej, windows_size, pol_order)
            rej_jf_c = savgol_filter(empf201903_rocjf_rej_c, windows_size, pol_order)
        if algos[5]==1:
            rej_dl1 = savgol_filter(empf201903_rocdl1_rej, windows_size, pol_order)
            rej_dl1_c = savgol_filter(empf201903_rocdl1_rej_c, windows_size, pol_order)

        
        if algos[0]==1:
            self.rej_ip2=rej_ip2d#_fitted
            self.rej_ip2_c=rej_ip2d_c#_fitted
        if algos[1]==1:
            self.rej_ip3=rej_ip3d#_fitted
            self.rej_ip3_c=rej_ip3d_c#_fitted
        if algos[2]==1:    
            self.rej_rnnip=rej_rnnip#_fitted
            self.rej_rnnip_c=rej_rnnip_c#_fitted
        if algos[3]==1:    
            self.rej_sv1=rej_sv1#_fitted
            self.rej_sv1_c=rej_sv1_c#_fitted
        if algos[4]==1:
            self.rej_jf=rej_jf#_fitted
            self.rej_jf_c=rej_jf_c#_fitted
        if algos[5]==1:
            self.rej_dl1=rej_dl1#_fitted
            self.rej_dl1_c=rej_dl1_c#_fitted

            self.x_ax=x_ax
            self.xdl1axis=xdl1axis
            self.xaxis=xaxis

In [404]:
def WP_plot(path,file_name,alg,verbose,flag,flav):#flag='jet' for eff_vs_jetpt, flag='H' for eff_vs_bhadrpt -- flav='B' or 'C'
    print(alg)
#    path='/home/salomon/Private/atlas/FTPF/Selector/DAOD_selector/'
#    file_name="debug_.root"

    file_empf201903=uproot.open(path+file_name)
    n_B=np.array(file_empf201903["nBjets"]).sum()
    n_C=np.array(file_empf201903["nCjets"]).sum()
    n_l=np.array(file_empf201903["nljets"]).sum() 

    if flag=='jet':
        hist=(((file_empf201903[alg+"_llr_jetpt_"+flav].values).T)[::-1])
    if flag=='H':
        hist=(((file_empf201903[alg+"_llr_jetpt_single"+flav].values).T)[::-1])

    nptbin=hist.shape[0]
    jet_pt=np.array(file_empf201903['pT_'+flav])
#    print(jet_pt)

    LLRalg_B=np.array(file_empf201903[alg+"_llr_B"])#+flav])
    LLRalg_B=np.delete(LLRalg_B, [0,len(LLRalg_B)-1])


    nbin=len(LLRalg_B)
    x_min=-20
    x_max=50
    xaxis=[x_min,x_max]
    Δx=x_max-x_min
    interval=Δx*(np.arange(0,nbin)/nbin)+x_min
    #print(len(interval)==nbin,nbin)

    if flav=='B':
        n_flav=n_B
    if flav=='C':
        n_flav=n_C

    N_LLR=integral(LLRalg_B,xaxis,xaxis,nbin)

    xM_alg=N_LLR/n_B#n_flav

    ptrange=[0,1000]
    pt_binning=[20,30,50,80,120,240]

    pt_step=nptbin/(ptrange[1]-ptrange[0])
    LLR_step=nbin/(xaxis[1]-xaxis[0])

    def pt_idx(pt):#pt in GeV
        return nptbin-int(pt*pt_step)

    def LLR_idx(LLR):
        return int((LLR-xaxis[0])*LLR_step)

    def r2_hist(h,pt_1,pt_2,LLR_1,LLR_2):
        return h[pt_idx(pt_2):pt_idx(pt_1),LLR_idx(LLR_1):LLR_idx(LLR_2)]

    def r1_hist(h,pt_1,pt_2):
        return h[pt_idx(pt_1):pt_idx(pt_2)]


    #Working points: 60%, 70%, 77% and 85% 
    eff_range=[0.6,0.7,0.77,0.85]
    eff_stack=np.array([])
    run=0
    for x in eff_range:

        #x=eff[1]
        if x<xM_alg:
            run=run+1
            WP=inverse(x,lambda t: integral(LLRalg_B/n_flav,[t,x_max],xaxis,nbin),xaxis,-1)[0]

            if verbose==True:
                print('Working point @',x,':', round(WP,2))


            eff_pt=np.array([])
            eff=np.array([])
            s_num=0
            s_den=0
            for k in range(0,len(pt_binning)-1):
    #            i1=nptbin-pt_idx(pt_binning[k])
    #            i2=nptbin-pt_idx(pt_binning[k+1])
    #            print(r1_hist(jet_pt,nptbin-pt_idx(pt_binning[k]),nptbin-pt_idx(pt_binning[k+1])).sum())
    #            eff_pt_den=np.append(eff_pt,(jet_pt[i1:i2]).sum())
                #print(n_B/N_LLR)
                nden=int((n_flav/N_LLR)*(r2_hist(hist,pt_binning[k],pt_binning[k+1],xaxis[0],xaxis[1]).sum()))
                nnum=int(r2_hist(hist,pt_binning[k],pt_binning[k+1],WP,xaxis[1]).sum())

                eff_pt_den=np.append(eff_pt,nden)
                eff_pt_num=np.append(eff_pt,nnum)
                s_num=s_num+nnum
                s_den=s_den+nden
                eff=np.append(eff,nnum/nden)
                if verbose==True:
                    print('efficiency in [',pt_binning[k],',',pt_binning[k+1],'] GeV:', round(eff[k],3), nnum, nden)

            print(s_num,s_den,n_B,s_num/s_den)

            if run==1:
                eff_m=np.append(eff[0],eff)
                eff_stack=eff_m
            if run>1:
                eff_m=np.append(eff[0],eff)
                eff_stack=np.vstack((eff_stack,eff_m))

    if run==1:
        for k in range(0,run):
            plt.step(pt_binning,eff_stack,label='WP: '+str(eff_range[k]))

    if run>1:
        for k in range(0,run):
            plt.step(pt_binning,eff_stack[k],label='WP: '+str(eff_range[k]))

    plt.xticks(pt_binning)
    #plt.xlim([0,1])
    plt.xlabel(flag+' pt [GeV]')
    if flav=='B':
        plt.ylim([0.4,1])
    plt.ylabel('efficiency')
    #plt.xscale('log')
    plt.legend(loc="lower right")

    plt.savefig(alg+'_'+flag+'_'+flav+'_WP.pdf')
    plt.show()

In [6]:
def roc_lrej(alg,alg_str,x_interval):
    pp = PdfPages('roc_lrej_'+alg_str+'.pdf')
    fig = plt.figure()
    plt.figure(figsize=(8,9))
#    plt.suptitle('AntiKt4EMPFlowJets')
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
    plt.subplot(gs[0])

#    plt.plot(alg.x_ax[0:len(alg.rej_ip2)],alg.rej_ip2, label='IP2D')
    plt.plot(alg.x_ax[0:len(alg.rej_ip3)],alg.rej_ip3, label='IP3D')
    plt.plot(alg.x_ax[0:len(alg.rej_rnnip)],alg.rej_rnnip, label='RNNIP')
    plt.plot(alg.x_ax[0:len(alg.rej_sv1)][0:len(alg.rej_rnnip)],alg.rej_sv1, label='SV1')
#    plt.plot(alg.x_ax[0:len(alg.rej_jf)],alg.rej_jf, label='JF')
    plt.plot(alg.x_ax[0:len(alg.rej_dl1)],alg.rej_dl1, label='DL1')

    plt.legend(loc="upper right")
    plt.xlim(x_interval)
    plt.ylim(1.,1e4)
    plt.yscale('log')
#    plt.xlabel('b-jet tagging efficiency')
    plt.ylabel('Light flavour rejection')

    plt.subplot(gs[1])
    den=alg.rej_rnnip
    s_den="RNNIP"
#    plt.plot(alg.x_ax[0:len(alg.rej_ip2)],alg.rej_ip2[0:len(alg.rej_ip2)]/den[0:len(alg.rej_ip2)], label='IP2D'+s_den)
    plt.plot(alg.x_ax[0:len(alg.rej_ip3)],alg.rej_ip3[0:len(alg.rej_ip3)]/den[0:len(alg.rej_ip3)], label='IP3D'+s_den)
    plt.plot(alg.x_ax[0:len(alg.rej_rnnip)],alg.rej_rnnip[0:len(alg.rej_rnnip)]/den[0:len(alg.rej_rnnip)], label='RNNIP'+s_den)
    plt.plot(alg.x_ax[0:len(alg.rej_sv1)],alg.rej_sv1[0:len(alg.rej_sv1)]/den[0:len(alg.rej_sv1)], label='SV1'+s_den)
#    plt.plot(alg.x_ax[0:len(alg.rej_jf)],alg.rej_jf[0:len(alg.rej_jf)]/den[0:len(alg.rej_jf)], label='JF'+s_den)
    len_min=min(len(alg.rej_rnnip),len(alg.rej_dl1))
    plt.plot(alg.x_ax[0:len_min],alg.rej_dl1[0:len_min]/den[0:len_min], label='DL1'+s_den)

    plt.xlabel('b-jet tagging efficiency')
    plt.ylabel('Ratio to '+s_den)
    plt.xlim(x_interval)
    plt.ylim(0.2,2)

    plt.savefig('roc_lrej_'+alg_str+'.pdf')
    pp.savefig()
    pp.close()

In [7]:
def roc_crej(alg,alg_str,x_interval):
    pp = PdfPages('roc_crej_'+alg_str+'.pdf')    
    fig = plt.figure()
    plt.figure(figsize=(8,9))
#    plt.suptitle('AntiKt4EMPFlowJets')
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
    plt.subplot(gs[0])

#    plt.plot(alg.x_ax[0:len(alg.rej_ip2_c)],alg.rej_ip2_c, label='IP2D')
    plt.plot(alg.x_ax[0:len(alg.rej_ip3_c)],alg.rej_ip3_c, label='IP3D')
    plt.plot(alg.x_ax[0:len(alg.rej_rnnip_c)],alg.rej_rnnip_c, label='RNNIP')
    plt.plot(alg.x_ax[0:len(alg.rej_sv1_c)][0:len(alg.rej_rnnip_c)],alg.rej_sv1_c, label='SV1')
#    plt.plot(alg.x_ax[0:len(alg.rej_jf_c)],alg.rej_jf_c, label='JF')
    plt.plot(alg.x_ax[0:len(alg.rej_dl1_c)],alg.rej_dl1_c, label='DL1')

    plt.legend(loc="upper right")
    plt.xlim(x_interval)
    plt.ylim(1.,1e3)
    plt.yscale('log')
#    plt.xlabel('b-jet tagging efficiency')
    plt.ylabel('c-jet rejection')

    plt.subplot(gs[1])
    den=alg.rej_rnnip_c
    s_den="RNNIP"
#    plt.plot(alg.x_ax[0:len(alg.rej_ip2_c)],alg.rej_ip2_c/den[0:len(alg.rej_ip2_c)], label='IP2D'+s_den)
    plt.plot(alg.x_ax[0:len(alg.rej_ip3_c)],alg.rej_ip3_c/den[0:len(alg.rej_ip3_c)], label='IP3D'+s_den)
    plt.plot(alg.x_ax[0:len(alg.rej_rnnip_c)],alg.rej_rnnip_c/den[0:len(alg.rej_rnnip_c)], label='RNNIP'+s_den)
    plt.plot(alg.x_ax[0:len(alg.rej_sv1_c)],alg.rej_sv1_c/den[0:len(alg.rej_sv1_c)], label='SV1'+s_den)
#    plt.plot(alg.x_ax[0:len(alg.rej_jf_c)],alg.rej_jf_c/den[0:len(alg.rej_jf_c)], label='JF'+s_den)
    len_min=min(len(alg.rej_rnnip),len(alg.rej_dl1))
    plt.plot(alg.x_ax[0:len_min],alg.rej_dl1_c[0:len_min]/den[0:len_min], label='DL1'+s_den)

    plt.xlabel('b-jet tagging efficiency')
    plt.ylabel('Ratio to '+s_den)
    plt.xlim(x_interval)
    plt.ylim(0.2,1.1)

    plt.savefig('roc_crej_'+alg_str+'.pdf')
    pp.savefig()
    pp.close()

In [1]:
def rec_lconf(alg_1,mode_1,alg_2,mode_2,x_interval):

        x_ax=alg_1.x_ax
        
        pp = PdfPages('CONFRONTO_IPxD_l_'+mode_1+'_'+mode_2+'.pdf')
        fig = plt.figure()
        plt.figure(figsize=(8,10))
        gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1]) 
        plt.subplot(gs[0])

#        plt.plot(x_ax[0:len(alg_1.rej_ip2)],alg_1.rej_ip2[0:len(alg_1.rej_ip2)], 'r', label='IP2D - '+mode_1, linestyle='dashed')
#        plt.plot(x_ax[0:len(alg_2.rej_ip2)],alg_2.rej_ip2[0:len(alg_2.rej_ip2)], 'r', label='IP2D - '+mode_2, linestyle='solid')
        plt.plot(x_ax[0:len(alg_1.rej_ip3)],alg_1.rej_ip3[0:len(alg_1.rej_ip3)], 'g', label='IP3D - '+mode_1, linestyle='dashed')
        plt.plot(x_ax[0:len(alg_2.rej_ip3)],alg_2.rej_ip3[0:len(alg_2.rej_ip3)], 'g', label='IP3D - '+mode_2, linestyle='solid')

        plt.legend(loc="upper right")
        plt.xlim(x_interval)
        plt.ylim(1.,1e4)
        plt.yscale('log')
#        plt.xlabel('b-jet tagging efficiency')
        plt.ylabel('Light flavour rejection')

#        plt.subplot(gs[1])
#        den=alg_1.rej_ip2
#        s_den="IP2D - "+mode_1
#        len_min=min(len(alg_1.rej_ip2),len(alg_2.rej_ip2))
#        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'r', linestyle='dashed')
#        plt.plot(x_ax[0:len_min],alg_2.rej_ip2[0:len_min]/den[0:len_min], 'r', label='IP2D - '+mode_2+'/'+s_den)

#        plt.legend(loc="upper right")
#        plt.xlabel('b-jet tagging efficiency')
#        plt.ylabel('Ratio to '+s_den)
#        plt.xlim(x_interval)
#        plt.ylim(0.8,1.2)

        plt.subplot(gs[1])
        den=alg_1.rej_ip3
        s_den="IP3D - "+mode_1
        len_min=min(len(alg_1.rej_ip3),len(alg_2.rej_ip3))
        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'g', linestyle='dashed')
        plt.plot(x_ax[0:len_min],alg_2.rej_ip3[0:len_min]/den[0:len_min], 'g', label='IP3D - '+mode_2+'/'+s_den)

        plt.legend(loc="upper right")
        plt.xlabel('b-jet tagging efficiency')
        plt.ylabel('Ratio to '+s_den)
        plt.xlim(x_interval)
#        plt.ylim(0.8,1.2)


        #plt.savefig('AntiKt4EMPFlowJets_BTagging201903_roc.pdf')
        pp.savefig()
        pp.close()
        
        
        pp = PdfPages('CONFRONTO_SV1-JF_Light_flavour_rej_'+mode_1+'_'+mode_2+'_multiplot.pdf')
        fig = plt.figure()
        plt.figure(figsize=(8,10))
        gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1]) 
        plt.subplot(gs[0])

        plt.plot(x_ax[0:len(alg_1.rej_sv1)],alg_1.rej_sv1[0:len(alg_1.rej_sv1)], 'r', label='SV1 - '+mode_1, linestyle='dashed')
        plt.plot(x_ax[0:len(alg_2.rej_sv1)],alg_2.rej_sv1[0:len(alg_2.rej_sv1)], 'r', label='SV1 - '+mode_2, linestyle='solid')
#        plt.plot(x_ax[0:len(alg_1.rej_jf)],alg_1.rej_jf[0:len(alg_1.rej_jf)], 'g', label='JF - '+mode_1, linestyle='dashed')
#        plt.plot(x_ax[0:len(alg_2.rej_jf)],alg_2.rej_jf[0:len(alg_2.rej_jf)], 'g', label='JF - '+mode_2, linestyle='solid')

        plt.legend(loc="upper right")
        plt.xlim(x_interval)
        plt.ylim(1.,1e4)
        plt.yscale('log')
#        plt.xlabel('b-jet tagging efficiency')
        plt.ylabel('Light flavour rejection')

        plt.subplot(gs[1])
        den=alg_1.rej_sv1
        s_den="SV1 - "+mode_1
        len_min=min(len(alg_1.rej_sv1),len(alg_2.rej_sv1))
        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'r', linestyle='dashed')
        plt.plot(x_ax[0:len_min],alg_2.rej_sv1[0:len_min]/den[0:len_min], 'r', label='SV1 - '+mode_2+'/'+s_den)

        plt.legend(loc="upper right")
        plt.xlabel('b-jet tagging efficiency')
        plt.ylabel('Ratio to '+s_den)
        plt.xlim(x_interval)
#        plt.ylim(0.8,1.2)
        
#        plt.subplot(gs[2])
#        den=alg_1.rej_jf
#        s_den="JF - "+mode_1
#        len_min=min(len(alg_1.rej_jf),len(alg_2.rej_jf))
#        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'g', linestyle='dashed')
#        plt.plot(x_ax[0:len_min],alg_2.rej_jf[0:len_min]/den[0:len_min], 'g', label='JF - '+mode_2+'/'+s_den)

 #       plt.legend(loc="upper right")
 #       plt.xlabel('b-jet tagging efficiency')
 #       plt.ylabel('Ratio to '+s_den)
 #       plt.xlim(x_interval)
 #       plt.ylim(0.2,5)

        #plt.savefig('AntiKt4EMPFlowJets_BTagging201903_roc.pdf')
        pp.savefig()
        pp.close()
   

        pp = PdfPages('CONFRONTO_RNNIP-DL1_Light_flavour_rej_'+mode_1+'_'+mode_2+'_multiplot.pdf')
        fig = plt.figure()
        plt.figure(figsize=(8,10))
        gs = gridspec.GridSpec(3, 1, height_ratios=[3, 1, 1]) 
        plt.subplot(gs[0])

        plt.plot(x_ax[0:len(alg_1.rej_rnnip)],alg_1.rej_rnnip[0:len(alg_1.rej_rnnip)], 'r', label='RNNIP - '+mode_1, linestyle='dashed')
        plt.plot(x_ax[0:len(alg_2.rej_rnnip)],alg_2.rej_rnnip[0:len(alg_2.rej_rnnip)], 'r', label='RNNIP - '+mode_2, linestyle='solid')
        plt.plot(x_ax[0:len(alg_1.rej_dl1)],alg_1.rej_dl1[0:len(alg_1.rej_dl1)], 'g', label='DL1 - '+mode_1, linestyle='dashed')
        plt.plot(x_ax[0:len(alg_2.rej_dl1)],alg_2.rej_dl1[0:len(alg_2.rej_dl1)], 'g', label='DL1 - '+mode_2, linestyle='solid')

        plt.legend(loc="upper right")
        plt.xlim(x_interval)
        plt.ylim(1.,1e4)
        plt.yscale('log')
#        plt.xlabel('b-jet tagging efficiency')
        plt.ylabel('Light flavour rejection')

        plt.subplot(gs[1])
        den=alg_1.rej_rnnip
        s_den="RNNIP - "+mode_1
        len_min=min(len(alg_1.rej_rnnip),len(alg_2.rej_rnnip))
        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'r', linestyle='dashed')
        plt.plot(x_ax[0:len_min],alg_2.rej_rnnip[0:len_min]/den[0:len_min], 'r', label='RNNIP- '+mode_2+'/'+s_den)

        plt.legend(loc="upper right")
#        plt.xlabel('b-jet tagging efficiency')
        plt.ylabel('Ratio to '+s_den)
        plt.xlim(x_interval)
#        plt.ylim(0.2,1.5)

        plt.subplot(gs[2])
        den=alg_1.rej_dl1
        s_den="DL1 - "+mode_1
        len_min=min(len(alg_1.rej_dl1),len(alg_2.rej_dl1))
        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'g', linestyle='dashed')
        plt.plot(x_ax[0:len_min],alg_2.rej_dl1[0:len_min]/den[0:len_min], 'g', label='DL1 - '+mode_2+'/'+s_den)

        plt.legend(loc="upper right")
        plt.xlabel('b-jet tagging efficiency')
        plt.ylabel('Ratio to '+s_den)
        plt.xlim(x_interval)
#        plt.ylim(0.2,1.5)

        #plt.savefig('AntiKt4EMPFlowJets_BTagging201903_roc.pdf')
        pp.savefig()
        pp.close()


In [9]:
def rec_cconf(alg_1,mode_1,alg_2,mode_2,x_interval):

        x_ax=alg_1.x_ax
        
        pp = PdfPages('CONFRONTO_IPxD_c_'+mode_1+'_'+mode_2+'.pdf')
        fig = plt.figure()
        plt.figure(figsize=(8,10))
        gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1]) 
        plt.subplot(gs[0])

#        plt.plot(x_ax[0:len(alg_1.rej_ip2_c)],alg_1.rej_ip2_c[0:len(alg_1.rej_ip2_c)], 'r', label='IP2D - '+mode_1, linestyle='dashed')
#        plt.plot(x_ax[0:len(alg_2.rej_ip2_c)],alg_2.rej_ip2_c[0:len(alg_2.rej_ip2_c)], 'r', label='IP2D - '+mode_2, linestyle='solid')
        plt.plot(x_ax[0:len(alg_1.rej_ip3_c)],alg_1.rej_ip3_c[0:len(alg_1.rej_ip3_c)], 'g', label='IP3D - '+mode_1, linestyle='dashed')
        plt.plot(x_ax[0:len(alg_2.rej_ip3_c)],alg_2.rej_ip3_c[0:len(alg_2.rej_ip3_c)], 'g', label='IP3D - '+mode_2, linestyle='solid')

        plt.legend(loc="upper right")
        plt.xlim(x_interval)
        plt.ylim(1.,1e2)
        plt.yscale('log')
        plt.ylabel('c-jet rejection')

#        plt.subplot(gs[1])
#        den=alg_1.rej_ip2_c
#        s_den="IP2D - "+mode_1
#        len_min=min(len(alg_1.rej_ip2_c),len(alg_2.rej_ip2_c))
#        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'r', linestyle='dashed')
#        plt.plot(x_ax[0:len_min],alg_2.rej_ip2_c[0:len_min]/den[0:len_min], 'r', label='IP2D - '+mode_2+'/'+s_den)

 #       plt.legend(loc="upper right")
 #       plt.ylabel('Ratio to '+s_den)
 #       plt.xlim(x_interval)
 #       plt.ylim(0.8,1.2)

        plt.subplot(gs[1])
        den=alg_1.rej_ip3_c
        s_den="IP3D - "+mode_1
        len_min=min(len(alg_1.rej_ip3_c),len(alg_2.rej_ip3_c))
        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'g', linestyle='dashed')
        plt.plot(x_ax[0:len_min],alg_2.rej_ip3_c[0:len_min]/den[0:len_min], 'g', label='IP3D - '+mode_2+'/'+s_den)

        plt.legend(loc="upper right")
        plt.xlabel('b-jet tagging efficiency')
        plt.ylabel('Ratio to '+s_den)
        plt.xlim(x_interval)
#        plt.ylim(0.8,1.2)


        #plt.savefig('AntiKt4EMPFlowJets_BTagging201903_roc.pdf')
        pp.savefig()
        pp.close()
        
        
        pp = PdfPages('CONFRONTO_SV1-JF_c-jet_rej_'+mode_1+'_'+mode_2+'_multiplot.pdf')
        fig = plt.figure()
        plt.figure(figsize=(8,10))
        gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1]) 
        plt.subplot(gs[0])

        plt.plot(x_ax[0:len(alg_1.rej_sv1_c)],alg_1.rej_sv1_c[0:len(alg_1.rej_sv1_c)], 'r', label='SV1 - '+mode_1, linestyle='dashed')
        plt.plot(x_ax[0:len(alg_2.rej_sv1_c)],alg_2.rej_sv1_c[0:len(alg_2.rej_sv1_c)], 'r', label='SV1 - '+mode_2, linestyle='solid')
#        plt.plot(x_ax[0:len(alg_1.rej_jf_c)],alg_1.rej_jf_c[0:len(alg_1.rej_jf_c)], 'g', label='JF - '+mode_1, linestyle='dashed')
#        plt.plot(x_ax[0:len(alg_2.rej_jf_c)],alg_2.rej_jf_c[0:len(alg_2.rej_jf_c)], 'g', label='JF - '+mode_2, linestyle='solid')

        plt.legend(loc="upper right")
        plt.xlim(x_interval)
        plt.ylim(1.,1e2)
        plt.yscale('log')
        plt.ylabel('c-jet rejection')

        plt.subplot(gs[1])
        den=alg_1.rej_sv1_c
        s_den="SV1 - "+mode_1
        len_min=min(len(alg_1.rej_sv1_c),len(alg_2.rej_sv1_c))
        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'r', linestyle='dashed')
        plt.plot(x_ax[0:len_min],alg_2.rej_sv1_c[0:len_min]/den[0:len_min], 'r', label='SV1 - '+mode_2+'/'+s_den)

        plt.legend(loc="upper right")
        plt.ylabel('Ratio to '+s_den)
        plt.xlim(x_interval)
#        plt.ylim(0.8,1.2)

#        plt.subplot(gs[2])
#        den=alg_1.rej_jf_c
#        s_den="JF - "+mode_1
#        len_min=min(len(alg_1.rej_jf_c),len(alg_2.rej_jf_c))
#        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'g', linestyle='dashed')
#        plt.plot(x_ax[0:len_min],alg_2.rej_jf_c[0:len_min]/den[0:len_min], 'g', label='JF - '+mode_2+'/'+s_den)

#        plt.legend(loc="upper right")
#        plt.xlabel('b-jet tagging efficiency')
#        plt.ylabel('Ratio to '+s_den)
#        plt.xlim(x_interval)
#        plt.ylim(0.2,5)

        #plt.savefig('AntiKt4EMPFlowJets_BTagging201903_roc.pdf')
        pp.savefig()
        pp.close()
        
        
        pp = PdfPages('CONFRONTO_RNNIP-DL1_c-jet_rej_'+mode_1+'_'+mode_2+'_multiplot.pdf')
        fig = plt.figure()
        plt.figure(figsize=(8,10))
        gs = gridspec.GridSpec(3, 1, height_ratios=[3, 1, 1]) 
        plt.subplot(gs[0])

        plt.plot(x_ax[0:len(alg_1.rej_rnnip_c)],alg_1.rej_rnnip_c[0:len(alg_1.rej_rnnip_c)], 'r', label='RNNIP - '+mode_1, linestyle='dashed')
        plt.plot(x_ax[0:len(alg_2.rej_rnnip_c)],alg_2.rej_rnnip_c[0:len(alg_2.rej_rnnip_c)], 'r', label='RNNIP - '+mode_2, linestyle='solid')
        plt.plot(x_ax[0:len(alg_1.rej_dl1_c)],alg_1.rej_dl1_c[0:len(alg_1.rej_dl1_c)], 'g', label='DL1 - '+mode_1, linestyle='dashed')
        plt.plot(x_ax[0:len(alg_2.rej_dl1_c)],alg_2.rej_dl1_c[0:len(alg_2.rej_dl1_c)], 'g', label='DL1 - '+mode_2, linestyle='solid')

        plt.legend(loc="upper right")
        plt.xlim(x_interval)
        plt.ylim(1.,1e3)
        plt.yscale('log')
        plt.ylabel('c-jet rejection')

        plt.subplot(gs[1])
        den=alg_1.rej_rnnip_c
        s_den="RNNIP - "+mode_1
        len_min=min(len(alg_1.rej_rnnip_c),len(alg_2.rej_rnnip_c))
        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'r', linestyle='dashed')
        plt.plot(x_ax[0:len_min],alg_2.rej_rnnip_c[0:len_min]/den[0:len_min], 'r', label='RNNIP- '+mode_2+'/'+s_den)

        plt.legend(loc="upper right")
        plt.ylabel('Ratio to '+s_den)
        plt.xlim(x_interval)
#        plt.ylim(0.2,1.5)

        plt.subplot(gs[2])
        den=alg_1.rej_dl1_c
        s_den="DL1 - "+mode_1
        len_min=min(len(alg_1.rej_dl1_c),len(alg_2.rej_dl1_c))
        plt.plot(x_ax[0:len(den)], np.array([1]*len(den)), 'g', linestyle='dashed')
        plt.plot(x_ax[0:len_min],alg_2.rej_dl1_c[0:len_min]/den[0:len_min], 'g', label='DL1 - '+mode_2+'/'+s_den)

        plt.legend(loc="upper right")
        plt.xlabel('b-jet tagging efficiency')
        plt.ylabel('Ratio to '+s_den)
        plt.xlim(x_interval)
#        plt.ylim(0.2,1.5)

        #plt.savefig('AntiKt4EMPFlowJets_BTagging201903_roc.pdf')
        pp.savefig()
        pp.close()
